In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
data=pd.read_csv('BMW sales data (2010-2024) (1).csv')

In [3]:
data.head()

,Model,Year,Region,Color,Fuel_Type,Transmission,Engine_Size_L,Mileage_KM,Price_USD,Sales_Volume,Sales_Classification
0,5 Series,2016,Asia,Red,Petrol,Manual,3.5,151748,98740,8300,High
1,i8,2013,North America,Red,Hybrid,Automatic,1.6,121671,79219,3428,Low
2,5 Series,2022,North America,Blue,Petrol,Automatic,4.5,10991,113265,6994,Low
3,X3,2024,Middle East,Blue,Petrol,Automatic,1.7,27255,60971,4047,Low
4,7 Series,2020,South America,Black,Diesel,Manual,2.1,122131,49898,3080,Low


In [4]:
data.tail()

,Model,Year,Region,Color,Fuel_Type,Transmission,Engine_Size_L,Mileage_KM,Price_USD,Sales_Volume,Sales_Classification
49995,i3,2014,Asia,Red,Hybrid,Manual,4.6,151030,42932,8182,High
49996,i3,2023,Middle East,Silver,Electric,Manual,4.2,147396,48714,9816,High
49997,5 Series,2010,Middle East,Red,Petrol,Automatic,4.5,174939,46126,8280,High
49998,i3,2020,Asia,White,Electric,Automatic,3.8,3379,58566,9486,High
49999,X1,2020,North America,Blue,Diesel,Manual,3.3,171003,77492,1764,Low


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Model                 50000 non-null  object 
 1   Year                  50000 non-null  int64  
 2   Region                50000 non-null  object 
 3   Color                 50000 non-null  object 
 4   Fuel_Type             50000 non-null  object 
 5   Transmission          50000 non-null  object 
 6   Engine_Size_L         50000 non-null  float64
 7   Mileage_KM            50000 non-null  int64  
 8   Price_USD             50000 non-null  int64  
 9   Sales_Volume          50000 non-null  int64  
 10  Sales_Classification  50000 non-null  object 
dtypes: float64(1), int64(4), object(6)
memory usage: 4.2+ MB


In [6]:
data.isna().sum()

Model                   0
Year                    0
Region                  0
Color                   0
Fuel_Type               0
Transmission            0
Engine_Size_L           0
Mileage_KM              0
Price_USD               0
Sales_Volume            0
Sales_Classification    0
dtype: int64

In [7]:
data.isnull().sum()

Model                   0
Year                    0
Region                  0
Color                   0
Fuel_Type               0
Transmission            0
Engine_Size_L           0
Mileage_KM              0
Price_USD               0
Sales_Volume            0
Sales_Classification    0
dtype: int64

In [8]:
data.duplicated().sum()

np.int64(0)

In [9]:
data['Sales_Classification'].value_counts()

Sales_Classification
Low     34754
High    15246
Name: count, dtype: int64

In [10]:
X=data.drop(columns=['Color','Model'])  
y=data['Sales_Classification']

In [11]:
from sklearn.preprocessing import LabelEncoder,StandardScaler
cat_col=['Region','Fuel_Type','Transmission','Sales_Classification']
for i in cat_col:
    le=LabelEncoder()
    X[i]=le.fit_transform(X[i])

In [12]:
num_col=['Year','Engine_Size_L','Mileage_KM','Price_USD','Sales_Volume']
scaler=StandardScaler()
X[num_col]=scaler.fit_transform(X[num_col])

In [13]:
from sklearn.model_selection import train_test_split
X_test,X_temp,y_test,y_temp=train_test_split(X,y,random_state=42,test_size=0.15,stratify=y)
X_train,X_val,y_train,y_val=train_test_split(X_temp,y_temp,random_state=42,test_size=0.1176,stratify=y_temp)

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
model=SVC(random_state=42,class_weight='balanced',probability=True,gamma='scale',kernel='rbf',C=1.0)
model.fit(X_train,y_train)

SVC(class_weight='balanced', probability=True, random_state=42)

In [15]:
y_pred_test=model.predict(X_test)
y_pred_val=model.predict(X_val)

In [16]:
from sklearn.metrics import accuracy_score,recall_score,recall_score,precision_score,classification_report,confusion_matrix,f1_score
def print_classification_metrics(y_true,y_pred,dataset_name="Dataset"):
    acc=accuracy_score(y_true,y_pred)
    prec=precision_score(y_true,y_pred,average='weighted')
    recc=recall_score(y_true,y_pred,average='weighted')
    f1=f1_score(y_true,y_pred,average='weighted')
    conf=confusion_matrix(y_true,y_pred)
    class_report=classification_report(y_true,y_pred)
    print(f"\n{dataset_name} Metrics:")
    print(f"Accuracy:{acc*100:.4f}%")
    print(f"Precision:{prec*100:.4f}%")
    print(f"Recall:{recc*100:.4f}%")
    print(f"F1 Score:{f1*100:.4f}%")
    print("Confusion Matrix:")
    print(conf)
    print("Classification Report:")
    print(class_report)

In [17]:
print_classification_metrics(y_val,y_pred_val,"Validation Set")
print_classification_metrics(y_test,y_pred_test,"Test Set")


Validation Set Metrics:
Accuracy:100.0000%
Precision:100.0000%
Recall:100.0000%
F1 Score:100.0000%
Confusion Matrix:
[[269   0]
 [  0 613]]
Classification Report:
              precision    recall  f1-score   support

        High       1.00      1.00      1.00       269
         Low       1.00      1.00      1.00       613

    accuracy                           1.00       882
   macro avg       1.00      1.00      1.00       882
weighted avg       1.00      1.00      1.00       882


Test Set Metrics:
Accuracy:100.0000%
Precision:100.0000%
Recall:100.0000%
F1 Score:100.0000%
Confusion Matrix:
[[12959     0]
 [    0 29541]]
Classification Report:
              precision    recall  f1-score   support

        High       1.00      1.00      1.00     12959
         Low       1.00      1.00      1.00     29541

    accuracy                           1.00     42500
   macro avg       1.00      1.00      1.00     42500
weighted avg       1.00      1.00      1.00     42500



In [18]:
from sklearn.model_selection import cross_val_score
model=SVC()
scores=cross_val_score(model,X,y,cv=5,scoring='accuracy')
print("Cross-validation scores:",scores)
print("Mean accuracy:",scores.mean())
print("Standard Deviation",scores.std())

Cross-validation scores: [1. 1. 1. 1. 1.]
Mean accuracy: 1.0
Standard Deviation 0.0
